# HW4A: The Local Model

### Bhaven Patel
### 4/16/2019

I worked with Anthony Rentsch, Lipika Ramaswamy, and Karina Huang on this homework.

My code can be found on my [Github](https://github.com/bhavenp/cs208/blob/master/homework/HW4/HW4_Bhaven_Patel.ipynb).

## Problem 1: Learning Conjunctions in the SQ Model

### (a)

### Centralized Version of SQ Model


For the centralized version of the SQ model, I chose to calculate $p_j = P[x[j]=0 \,\wedge\, y=1]$ for $j=1,...,d$. 
To do this, I create a "conjunction matrix" where the element at the $i$-th row ($i=1,...,n$) and the $j$-th column ($j=1,...,d$) contains an indicator as to whether $x_{ij}=0 \wedge y_i=1$ in the original dataset. $p_j$ is just the mean of the $j$-th column in the conjunction matrix.

Then, laplace noise is added to $p_j$ with a scale equal to $\dfrac{GS}{\tilde\epsilon}$, where the global sensitivity $GS=\dfrac{1}{n}$ and $\tilde\epsilon= \dfrac{\epsilon}{d}$. The $GS=\dfrac{1}{n}$ because changing one value ($0\rightarrow 1$ or $1\rightarrow 0$) in the column $j$ changes $p_j$ by $\dfrac{1}{n}$. Thus, every $p_j$ has a differentially private release $\hat p_j$
$$
\hat p_j = p_j + Lap\left(\dfrac{d}{n\tilde\epsilon}  \right)
$$

Below are the helper functions we generally use.

In [1]:
rm(list=ls())		# Remove any objects in memory

# Random draw from Laplace distribution
#
# mu numeric, center of the distribution
# b numeric, spread
# size integer, number of draws
# 
# return Random draws from Laplace distribution
# example:
# 
# rlap(size=1000)

rlap = function(mu=0, b=1, size=1) {
    p <- runif(size) - 0.5
    draws <- mu - b * sgn(p) * log(1 - 2 * abs(p))
    return(draws)
}

# Sign function
# 
# Function to determine what the sign of the passed values should be.
#
# x numeric, value or vector or values
# return The sign of passed values
# example:
#
# sgn(rnorm(10))

sgn <- function(x) {
    return(ifelse(x < 0, -1, 1))
}

In [2]:
##function to create the matrix that holds an indicator if x_j==0 & y==1
createConjunctionMat <- function(xData, yData){
    #create matrix to hold indicator if x_j==0 & y==1
    result_matrix = matrix(0, nrow=nrow(xData), ncol=ncol(xData));
    for(i in 1:nrow(xData)){
        if(yData[i] == 1){ #only need to consider row if y=1
            result_matrix[i, ] <- (xData[i, ] == 0); #check if x_j == 0
        }
    }
    return(result_matrix);
}

#function to calculate DP-releases for each probability
probRelease <- function(xMat, epsilon=1.0){
    probs <- colMeans(xMat); #calculate true probabilities
    sensitivity <- 1 / nrow(xMat); #sensitivity is 1/n
    scale <- sensitivity / epsilon;
    dpProbs <- probs + rlap(mu=0, b=scale, size=length(probs)); #add laplace noise to the true probabilities

	return(list(release=dpProbs, true=probs) );
}

#function that ties together the different parts for doing a DP release of the probabilities for a 
## xData: matrix of {0,1}
## yData: vector of {0,1}, same length as number of rows in xData
## epsilon: total privacy-loss parameter. This will get split up by the number of columns in xData that 
##          we must release probabilities for
## returns a list containing a vector of the indices corresponding to the columns of xData that predict yData well
## and a vector of the DP released probabilities calculated for each column

centrlDP_SQAlg <- function(xData, yData, totEpsilon=1.0, threshold=1e-4){
    pMatrix <- createConjunctionMat(xData=xData, yData=yData); #create conjunction matrix
#     print(totEpsilon/ncol(xData))
    dpRelease <- probRelease(pMatrix, epsilon = totEpsilon/ncol(xData) ); #get DP release of probabilities
#     cat(dpRelease$true);
#     cat(dpRelease$release);
    indices <- which(dpRelease$release < threshold); #get indices with probability less than threshold
    return(list(indices=indices, probs=dpRelease$release) );
}

In [3]:
#read in the test data
mydata <- read.csv('../../data/hw4testdata.csv');
# mydata[0:10, ]

#get set of features to use as predictors
set.seed(42);
centrlDP_SQAlg(xData = mydata[, 1:10], yData = mydata[['y']], totEpsilon = 1.0, threshold = .01)

$indices
[1] 1 2 3

$probs
 [1]  1.769678e-04  2.072671e-04 -5.581285e-05  6.207814e-02  6.055334e-02
 [6]  6.118389e-02  6.161409e-02  6.166882e-02  6.089769e-02  6.109279e-02

### Local Model

### describe the local model set-up and how I did the correction

For the local model


In [4]:
#local release mechanism works for x as a vector. Adjusted from J. Honaker's localRelease.ipynb.
##
## x: vector of {0,1} for which local release must be performed
## values: vector of length two containing the possible values in vector 'x'
## epsilon: privacy-loss parameter to use for flipping values in 'x'
## returns a vector

localReleaseVec <- function(x, values=c(0,1), epsilon){
	draws <- runif(n=length(x), min=0, max=1); #get number of draws equal to length of the vector
	cutoff <- 1/(1+exp(epsilon));
    release <- x; #make a copy of the vector x
    for(i in 1:length(x)){
        if(draws[i] < cutoff){ #we are going to flip the our value
            release[i] <- values[ !values %in% x[i] ] ; #create flag with (!values %in% x) for opposite value of 'x'
        }
    } 
    return(release);
}


#function that ties together the different parts for doing a DP release of the probabilities for a local model
## xData: matrix of {0,1}
## yData: vector of {0,1}, same length as number of rows in xData
## epsilon: total privacy-loss parameter. This will get split up by the number of columns in xData that 
##          we must release probabilities for
## returns a list containing a vector of the indices corresponding to the columns of xData that predict yData well
## and a vector of the DP released probabilities calculated for each column 

localDP_SQAlg <- function(xData, yData, totEpsilon=1.0, threshold=1e-4){
#     cat(paste(Sys.time(),"\n") )
    cjMatrix <- createConjunctionMat(xData=xData, yData=yData);#create conjunction matrix
#     cat(colMeans(cjMatrix));
    
    epsSplit <- totEpsilon / ncol(xData); #divide the total epsilon by the number of features
    #perform local release of conjunction matrix 
    lrMatrix <- cjMatrix; #create a copy of the conjunction matrix
    for(r in 1:nrow(cjMatrix)){
        lrMatrix[r, ] <- localReleaseVec(x=cjMatrix[r, ], values = c(0,1), epsilon = epsSplit);
    }
    

    dpSums <- colSums(lrMatrix); #get sums of the local release of the conjunction matrix
#     cat('\n', dpSums);
    #perform correction
    inflation <- (exp(epsSplit) + 1) / (exp(epsSplit) - 1); #coefficient to multiply each sum by
    n <- nrow(xData); #get number of rows
    additive <- -n / (exp(epsSplit) - 1); #additive factor to apply for because data is {0,1}
#     print(paste(inflation, additive))
    dpProbs <- (dpSums * inflation + additive) / n;
#     cat('\n', dpProbs);
    indices <- which(dpProbs < threshold); #get indices with probability less than threshold
    return(list(indices=indices, probs=dpProbs));
}

In [5]:
set.seed(42);
mydata <- read.csv('../../data/hw4testdata.csv');
localDP_SQAlg(xData = mydata[, 1:10], yData = mydata[['y']], totEpsilon = 1.0, threshold = 0.01)

$indices
[1] 1 2 3

$probs
 [1] -0.01002460 -0.05586276 -0.01162593  0.08345322  0.07224389  0.05022556
 [7]  0.08285272  0.08985856  0.05102623  0.07584689

### (b)

### Centralized DP algorithm

I will refer to the $d$ columns in each dataset $x_i$ as features.<br>
$P[\hat S \supseteq S]$ is the probability that set $S$ contains a feature $j$ that $\hat S$ does not. This equivalent to determining the probability that at least one of the features $j \in S$ has DP-released $\hat p_j$ that is greater than the threshold $t$ we specify. Thus, we get
$$
P[\hat S \supseteq S] = \sum_{j\in S} P[\hat p_j > t]
$$

For the centralized DP SQ algorithm, I chose $\hat p_j = p_j + Lap\left(\dfrac{d}{n\tilde\epsilon}\right)$. Additionally, we know that for each $j \in S$, we have $p_j =0$. Thus, we get
$$\begin{align}
P[\hat S \supseteq S] &= \sum_{j\in S} P[p_j + Lap\left(\dfrac{d}{n\tilde\epsilon}\right) > t]\\
&= \sum_{j\in S} P[0 + Lap\left(\dfrac{d}{n\tilde\epsilon}\right) > t]
\end{align}$$

This can be equation can be expanded to be in terms of $t$, $n$, $\epsilon$, and $|S|$ because we can expand the probability that a value chosen from the Laplace distribution is greater than $t$ and $\sum_{j\in S}=|S|$:
$$\begin{align}
&= \sum_{j\in S} P[0 + Lap\left(\dfrac{d}{n\tilde\epsilon}\right) > t] \\
&= |S| \cdot \int_t^{\infty} \dfrac{e^{(-|y|\cdot n\epsilon / d)}\cdot n\epsilon}{2d} dy\\
\end{align}$$

Now, if we say that $P[\hat S \supseteq S] \leq 0.1$, we can find an upper bound on what the threshold $t$ should be:
$$\begin{align}
P[\hat S \supseteq S] &\leq 0.1\\
\\
|S| \cdot \int_t^{\infty} \dfrac{e^{(-|y|\cdot n\epsilon / d)}\cdot n\epsilon}{2d} dy &\leq 0.1\\
\\
|S|\dfrac{n\epsilon}{2d} \cdot \int_t^{\infty} e^{(-|y|\cdot n\epsilon / d)} dy &\leq 0.1\\
\\
\\
\int_t^{\infty} e^{(-|y|\cdot n\epsilon / d)} dy &\leq \dfrac{(0.1)\cdot 2d}{|S|n\epsilon}\\
\\
\left[\dfrac{-d}{n\epsilon} e^{(-y\cdot n\epsilon / d)} \right]_t^{\infty} &\leq \dfrac{(0.1)\cdot 2d}{|S|n\epsilon}\\
\\
\dfrac{-d}{n\epsilon} \left[ e^{(-\infty \cdot n\epsilon / d)} - e^{(-t \cdot n\epsilon / d)} \right] &\leq \dfrac{(0.1)\cdot 2d}{|S|n\epsilon}\\
\\
-1 \left[ 0 - e^{(-t \cdot n\epsilon / d)} \right] &\leq \dfrac{(0.2)}{|S|}\\
\\
e^{(-t \cdot n\epsilon / d)} &\leq \dfrac{(0.2)}{|S|}\\
\\
-t  &\leq \dfrac{d}{n\epsilon} \log\left(\dfrac{(0.2)}{|S|} \right)\\
\\
t  &\leq \dfrac{-d}{n\epsilon} \log\left(\dfrac{(0.2)}{|S|} \right)\\
\end{align}$$

Thus, for the centralized DP SQ algorithm, we find that $t  \leq \dfrac{-d}{n\epsilon} \log\left(\dfrac{(0.2)}{|S|} \right)$ if we want $P[\hat S \supseteq S] \leq 0.1$.

### (c)

### Centralized DP SQ algorithm

From part (b), we know that the upper bound on the threshold $t$ is $t \leq \dfrac{-d}{n\epsilon} \log\left(\dfrac{(0.2)}{|S|} \right)$. For the *CaPUMS5full.csv* data, we can calculate the upper bound using $n=1223992$, $d=10$, $\epsilon = 1.0$, and $|S|=d=11$.
$$\begin{align}
t &= \dfrac{-d}{n\epsilon} \log\left(\dfrac{(0.2)}{|S|} \right)\\
&= \dfrac{10}{(1223992)(1.0)} \log\left(\dfrac{(0.2)}{10} \right)\\
&= 3.20 \cdot 10^{-5}
\end{align}$$

Thus for the *CaPUMS5full.csv* dataset, the upper bound on the threshold $t$ is $3.20 \cdot 10^{-5}$. I will use this threshold to show that my centralized DP SQ algorithm works on the *CaPUMS5full.csv* dataset.

In [3]:
#read in the test data
caPUMS <- read.csv('../../data/CaPUMS5full.csv');
print(paste(nrow(caPUMS), " rows in CA PUMS"));
caPUMS[1:3, ]

[1] "1223992  rows in CA PUMS"


sex,married,black,asian,collegedegree,employed,militaryservice,uscitizen,disability,englishability,blackfemale,targetted
1,1,0,0,0,1,0,1,0,1,0,0
1,1,0,0,0,0,0,1,0,1,0,0
0,1,0,0,0,0,1,1,0,1,0,0


In [22]:
#get set of features to use as predictors
cDPInd <- centrlDP_SQAlg(xData = caPUMS[, 1:10], yData = caPUMS[['targetted']], totEpsilon = 1.0, threshold = 3.2e-5);
print(cDPInd$indices) #print the indices chosen as predictors for targetted
print(colnames(caPUMS)[cDPInd$indices]) #print the colnames from the original dataset that the indices refer to

[1]  6  8 10
[1] "employed"       "uscitizen"      "englishability"


Using the threshold $t$ I calculated, we see that the columns *employed*, *uscitizen*, and *englishability* are the columns chosen for the set $\hat S$.

I will now use bootstraps of various sizes $n$ to compare the false positive rates and false negative rates as a function of $n$ for my centralized DP SQ algorithm. For each $n$, I will generate a bootstrapped sample and generate 10 DP releases for the set of features $\hat S$ that predict **targetted=1**; I will calculate a false positive rate and false negative rate by predicting the value of **targetted** using the conjunction of the features included in $\hat S$ for each of the 10 DP releases.

In [6]:
#function to calculate false positive and false negative rates given true and predicted data
calcFPandFN <- function(yTrue, yPreds){
    numPos <- sum(yTrue); #number of positive results is just sum of the vector
    numNeg <- length(yTrue) - numPos;
    
    #calculate the false negative
    posInd <- which(yTrue == 1); #get indices of positive results
    tps <- sum( yTrue[posInd] == yPreds[posInd]);
    fnRate <- 1 - tps/numPos; #FN-rate is 1 - TP-rate
    
    #calculate the false positive
    negInd <- which(yTrue == 0); #get indices of negative results
    tns <- sum( yTrue[negInd] == yPreds[negInd]);
    fpRate <- 1 - tns/numNeg; #FP-rate is 1 - TN-rate
    
    return(list(fp=fpRate, fn=fnRate));
}

In [ ]:
# set.seed(24);
sampSizes <- c(1e2, 1e3, 5e3, 1e4, 5e4, 1e5, 5e5, 1e6); #size of datasets
numSims <- 10; #number of simulations per bootstrap

numRows <- length(sampSizes) * numSims
resultsMatrix <- matrix(NA, nrow = numRows, ncol = 3);

r = 1; #row counter
for(sSize in sampSizes){
    #sample the indices from the dataset
    sample_ind <- sample(x=1:nrow(caPUMS), size=sSize, replace=TRUE);
    bootstrap <- caPUMS[sample_ind, ]; #create a bootstrap sample
            
    for(i in 1:numSims){
        cDPInd <- centrlDP_SQAlg(xData = bootstrap[, 1:10], yData = bootstrap[['targetted']], 
                                 totEpsilon = 1.0, threshold = 3.2e-5);
        #check if no columns were selected
        while(length(cDPInd$indices) == 0){ #re-run release if feature set is empty
            cDPInd <- centrlDP_SQAlg(xData = bootstrap[, 1:10], yData = bootstrap[['targetted']], 
                                 totEpsilon = 1.0, threshold = 3.2e-5);
        }
#         print(cDPInd)
        
        #get the predictors specified by the bootstrap
        bootstrapPreds <- bootstrap[, cDPInd$indices];
        
        if(length(cDPInd$indices) > 1){
            #get predictions for bootstrapped sample. If conjunction of all predictors is 1,
            #then prediction is 1.
            yPreds <- (rowSums(bootstrapPreds) == length(cDPInd$indices) );
        }else{ #handle case when there is just one predictor in set
            yPreds <- bootstrapPreds;
        }
        
        #get the FP and FN rates
        stats <- calcFPandFN(yTrue=bootstrap[['targetted']], yPreds=yPreds);
        resultsMatrix[r, ] <- c(sSize, stats$fp, stats$fn)
        r = r+1; #increment row counter
    }
}

In [ ]:
final_results <- as.data.frame(resultsMatrix);
colnames(final_results) <- c("n", "FPR", "FNR");
write.csv(final_results, file='ctlDP_bootstrap.csv', row.names=FALSE);

### Local DP SQ algorithm, fill the rest out using optimal threshold

From part (b), we know that the upper bound on the threshold $t$ is $t \leq \dfrac{-d}{n\epsilon} \log\left(\dfrac{(0.2)}{|S|} \right)$. For the *CaPUMS5full.csv* data, we can calculate the upper bound using $n=1223992$, $d=10$, $\epsilon = 1.0$, and $|S|=d=11$.
$$\begin{align}
t &= \dfrac{-d}{n\epsilon} \log\left(\dfrac{(0.2)}{|S|} \right)\\
&= \dfrac{10}{(1223992)(1.0)} \log\left(\dfrac{(0.2)}{10} \right)\\
&= 3.20 \cdot 10^{-5}
\end{align}$$

Thus for the *CaPUMS5full.csv* dataset, the upper bound on the threshold $t$ is $3.20 \cdot 10^{-5}$. I will use this threshold to show that my centralized DP SQ algorithm works on the *CaPUMS5full.csv* dataset.

In [33]:
localRelInd <- localDP_SQAlg(xData = mydata[, 1:10], yData = mydata[['targetted']], totEpsilon = 1.0, threshold = 1e-2);
print(localRelInd)


 596597 588430 596150 596116 589969 581356 592187 581080 594653 581563
 0.2481711 0.1146113 0.240861 0.240305 0.1397795 -0.001074012 0.1760517 -0.005587603 0.2163797 0.002311182[1]  6  8 10


In [32]:
localReleaseVec(x = c(0,1,0), epsilon = 1.0, values = c(0,1))

[1] 0 1 0